In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon

print('Loading Files')
cellrebel_pd =pd.read_csv('csv_polygon/GBK_Senayan.csv')
cellrebel_gpd = gpd.GeoDataFrame(cellrebel_pd, geometry=gpd.GeoSeries.from_wkt(cellrebel_pd['WKT']), crs=4326)

xmin, ymin, xmax, ymax = cellrebel_gpd.total_bounds

length =  0.0008985 / 20  ### (the distance for 100 metres is 0.0008985, so if you want to get 250 metres, need to multiply 2.5)
wide =  0.0008985 / 20 ### (the distance for 100 metres is 0.0008985, so if you want to get 250 metres, need to multiply 2.5)

print('Processing')
cols = list(np.arange(xmin, xmax + wide, wide))
rows = list(np.arange(ymin, ymax + length, length))

polygons = []
for x in cols[:-1]: ### [:-1] these are used stop -1 of position from the end of list 
    for y in rows[:-1]:
        polygons.append(Polygon([(x,y), (x+wide, y), (x+wide, y+length), (x, y+length)]))

grid = gpd.GeoDataFrame({'geometry':polygons}, crs=4326)

grid = grid.sjoin(cellrebel_gpd, how='inner', predicate='intersects')
grid.reset_index(inplace=True)
grid.rename(columns={'index':'id'}, inplace = True)

grid['geometry'] = grid['geometry'].astype(str)
grid.drop(columns=['id','index_right', 'WKT'], inplace=True)
# grid.rename(columns={'geometry':'WKT'}, inplace=True)

print('Saving to File....')
grid['partition_column'] = grid.index % 15

grid.to_parquet('csv_polygon/5x5grid_GBK.parquet', index=False)
#grid.to_csv('grid_folder/40x40grid_alljabo.csv', index=False)
#ddf = da.from_pandas(grid, chunksize=3000000)
#ddf.to_parquet('/home/nivag/grid_folder/grid_40x40/')


Loading Files
Processing
Saving to File....


/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
